In [79]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score, recall_score, precision_score, accuracy_score
import numpy as np

from sklearn.ensemble import IsolationForest
from joblib import dump, load
import pandas as pd 
import plotly.express as px

In [80]:
dfo = pd.read_csv("..\\Data\\SmA-Four-Tank-Batch-Process_V2.csv",sep=';')
dfo['timestamp']=pd.to_datetime(dfo['timestamp'])
#df_day=dfo.copy()
#df_day['h']=df_day['timestamp'].dt.hour

In [81]:
df=dfo.sample(50000)
#df=dfo.copy()
df=df.drop(columns=["timestamp"])
c=list(df.columns)
c[0]="Dev"
df.columns=c
df=df[df['Dev']!=0]
df['Dev']=df['Dev']!=1
df['Dev']=df['Dev'].astype('int')
df_clean=df.copy()

In [82]:
df_clean.to_csv('..\\Data\\data_01_clean.csv')
X_train,X_test,y_train,y_test=train_test_split(df_clean.drop(columns=['Dev']),df_clean['Dev'],shuffle=False,random_state=42)

In [83]:
model=RandomForestClassifier()#n_estimators=100)
model.fit(X_train,y_train)

RandomForestClassifier()

In [84]:
y_pred_train=model.predict(X_train)
print(f'R2:{r2_score(y_train,y_pred_train)}\nAccuracy: {accuracy_score(y_train,y_pred_train)}\nRecall: {recall_score(y_train,y_pred_train)}\nPrecision: {precision_score(y_train,y_pred_train)}')

tn, fp, fn, tp = confusion_matrix(y_train,y_pred).ravel()
print(f'TN: {tn}')
print(f'FP: {fp}')
print(f'FN: {fn}')
print(f'TP: {tp}')

R2:1.0
Accuracy: 1.0
Recall: 1.0
Precision: 1.0


In [85]:

tn, fp, fn, tp = confusion_matrix(y_train,y_pred).ravel()
print(f'TN: {tn}')
print(f'FP: {fp}')
print(f'FN: {fn}')
print(f'TP: {tp}')

TN: 4929
FP: 0
FN: 0
TP: 32568


In [86]:
y_pred=model.predict(X_test)
print(f'R2:{r2_score(y_test,y_pred)}\nAccuracy: {accuracy_score(y_test,y_pred)}\nRecall: {recall_score(y_test,y_pred)}\nPrecision: {precision_score(y_test,y_pred)}')

R2:0.6884391850228933
Accuracy: 0.9650372029762381
Recall: 0.9858572871705391
Precision: 0.9743147576692684


In [87]:
y_pred=model.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
print(f'TN: {tn}')
print(f'FP: {fp}')
print(f'FN: {fn}')
print(f'TP: {tp}')

TN: 1327
FP: 283
FN: 154
TP: 10735
